### 写个demo试试技术路线

In [1]:
import json
from pathlib import Path
import numpy as np
import pandas as pd
# from scipy import sparse
from tqdm import tqdm
import torch

pd.options.display.width = 180
pd.options.display.max_colwidth = 120
MAX_FILE_NUM = 1000

读取json文件的函数 read_notebook

In [2]:
def read_notebook(path):
    df = pd.read_json(
            path,
            dtype={'cell_type':'category' , 'source':'str'}
    ).assign(id=path.stem).rename_axis('cell_id')
    return df

读部分试试，读太多算不过来

In [3]:
paths_train = list(Path('train').glob('*json'))[: MAX_FILE_NUM]
paths_train

[WindowsPath('train/00001756c60be8.json'),
 WindowsPath('train/00015c83e2717b.json'),
 WindowsPath('train/0001bdd4021779.json'),
 WindowsPath('train/0001daf4c2c76d.json'),
 WindowsPath('train/0002115f48f982.json'),
 WindowsPath('train/00035108e64677.json'),
 WindowsPath('train/00038c2941faa0.json'),
 WindowsPath('train/00039fc77a9dd1.json'),
 WindowsPath('train/00058dc97eae4b.json'),
 WindowsPath('train/000597ac4c6700.json'),
 WindowsPath('train/000624d747afd3.json'),
 WindowsPath('train/00062ab8487156.json'),
 WindowsPath('train/000757b90aaca0.json'),
 WindowsPath('train/0007f21ee357b5.json'),
 WindowsPath('train/000890decea38e.json'),
 WindowsPath('train/0008ba887b3817.json'),
 WindowsPath('train/0009acaa9aa47e.json'),
 WindowsPath('train/000a2f5243e1ca.json'),
 WindowsPath('train/000a4651cce8f4.json'),
 WindowsPath('train/000af347c31e72.json'),
 WindowsPath('train/000b8e6d58544b.json'),
 WindowsPath('train/000bbb79a2fe3c.json'),
 WindowsPath('train/000c0a9b2fef4d.json'),
 WindowsPat

In [4]:
df = read_notebook(paths_train[0])
# df[[df["cell_type"] == "code"]]
allCode = df[(df["cell_type"] == "code").tolist()]
allCode

,cell_type,source,id
cell_id,,,
1862f0a6,code,# This Python 3 environment comes with many helpful analytics libraries installed\n# It is defined by the kaggle/pyt...,00001756c60be8
2a9e43d6,code,"import numpy as np\nimport pandas as pd\nimport random\n\nfrom sklearn.model_selection import train_test_split, cros...",00001756c60be8
038b763d,code,import warnings\nwarnings.filterwarnings('ignore'),00001756c60be8
2eefe0ef,code,matplotlib.rcParams.update({'font.size': 14}),00001756c60be8
0beab1cd,code,"def evaluate_preds(train_true_values, train_pred_values, test_true_values, test_pred_values):\n print(""Train R2:\...",00001756c60be8
9a78ab76,code,TRAIN_DATASET_PATH = '/kaggle/input/real-estate-price-prediction-moscow/train.csv'\nTEST_DATASET_PATH = '/kaggle/inp...,00001756c60be8
ebe125d5,code,train_df = pd.read_csv(TRAIN_DATASET_PATH)\ntrain_df.tail(),00001756c60be8
d9dced8b,code,train_df.dtypes,00001756c60be8
86497fe1,code,num_feat = list(train_df.select_dtypes(exclude='object').columns)\nobj_feat = list(train_df.select_dtypes(include='o...,00001756c60be8


In [5]:
import re
one_code_cell = allCode.iloc[0 , 1]
# one_code_cell = one_code_cell.replace("\n" , " ")
one_code_cell = re.sub(r"^#.*\n" , " " , one_code_cell , flags=re.MULTILINE)
one_code_cell

"   \nimport numpy as np # linear algebra\nimport pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)\n\n  \nimport os\nfor dirname, _, filenames in os.walk('/kaggle/input'):\n    for filename in filenames:\n        print(os.path.join(dirname, filename))\n\n # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session"

In [6]:
code_text = allCode.groupby("id")["source"].apply(lambda x: x.str.cat(sep=" ")).values
code_text

array(['# This Python 3 environment comes with many helpful analytics libraries installed\n# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python\n# For example, here\'s several helpful packages to load\n\nimport numpy as np # linear algebra\nimport pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)\n\n# Input data files are available in the read-only "../input/" directory\n# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory\n\nimport os\nfor dirname, _, filenames in os.walk(\'/kaggle/input\'):\n    for filename in filenames:\n        print(os.path.join(dirname, filename))\n\n# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" \n# You can also write temporary files to /kaggle/temp/, but they won\'t be saved outside of the current session import numpy as np\nimport pandas as pd\n

In [7]:
clean_code = re.sub(r"^#.*\n" , " " , code_text[0] , flags=re.MULTILINE) #第一次去除 #类注释
# clean_code = re.sub(r"^\s#.*\s*" , " " , code_text[0] , flags=re.MULTILINE)
clean_code = re.sub(r'""".+"""', ' ', clean_code) # 去除 “”“”“”类型注释
clean_code = re.sub(r' +', ' ', clean_code) # 去除多个空格那类型
clean_code = clean_code.replace("\n" , " ") # 去除换行符
clean_code

'  import numpy as np # linear algebra import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)    import os for dirname, _, filenames in os.walk(\'/kaggle/input\'):  for filename in filenames:  print(os.path.join(dirname, filename))   import pandas as pd import random  from sklearn.model_selection import train_test_split, cross_val_score from sklearn.preprocessing import StandardScaler, RobustScaler from catboost import CatBoostRegressor from sklearn.ensemble import RandomForestRegressor from sklearn.metrics import r2_score as r2 from sklearn.model_selection import KFold, GridSearchCV  from datetime import datetime  import matplotlib import matplotlib.pyplot as plt import seaborn as sns %matplotlib inline import warnings warnings.filterwarnings(\'ignore\') matplotlib.rcParams.update({\'font.size\': 14}) def evaluate_preds(train_true_values, train_pred_values, test_true_values, test_pred_values):  print("Train R2:\\t" + str(round(r2(train_true_values, train_pred_values), 

In [36]:
from transformers import RobertaTokenizer , T5ForConditionalGeneration
tokenizer = RobertaTokenizer.from_pretrained('Salesforce/codet5-base-multi-sum')
model = T5ForConditionalGeneration.from_pretrained('Salesforce/codet5-base-multi-sum')

inputs_ids = tokenizer(clean_code , return_tensors="pt")["input_ids"]
generated_ids = model.generate(inputs_ids , max_length=50)

tokenizer.decode(generated_ids[0], skip_special_tokens=True)

Token indices sequence length is longer than the specified maximum sequence length for this model (4048 > 512). Running this sequence through the model will result in indexing errors


'Function to plot the real - state data.'

看上去效果并不是很好，后面打算对每一个code cell单独使用summary

In [8]:
allCode #这个我就感觉不用清洗了，最多就是去除一下 \n 这种

,cell_type,source,id
cell_id,,,
1862f0a6,code,# This Python 3 environment comes with many helpful analytics libraries installed\n# It is defined by the kaggle/pyt...,00001756c60be8
2a9e43d6,code,"import numpy as np\nimport pandas as pd\nimport random\n\nfrom sklearn.model_selection import train_test_split, cros...",00001756c60be8
038b763d,code,import warnings\nwarnings.filterwarnings('ignore'),00001756c60be8
2eefe0ef,code,matplotlib.rcParams.update({'font.size': 14}),00001756c60be8
0beab1cd,code,"def evaluate_preds(train_true_values, train_pred_values, test_true_values, test_pred_values):\n print(""Train R2:\...",00001756c60be8
9a78ab76,code,TRAIN_DATASET_PATH = '/kaggle/input/real-estate-price-prediction-moscow/train.csv'\nTEST_DATASET_PATH = '/kaggle/inp...,00001756c60be8
ebe125d5,code,train_df = pd.read_csv(TRAIN_DATASET_PATH)\ntrain_df.tail(),00001756c60be8
d9dced8b,code,train_df.dtypes,00001756c60be8
86497fe1,code,num_feat = list(train_df.select_dtypes(exclude='object').columns)\nobj_feat = list(train_df.select_dtypes(include='o...,00001756c60be8


In [9]:
allCode.loc[: , "source"] = allCode["source"].apply(lambda text: text.replace("\n" , " "))
allCode

e:\PyCharm\python3.9.6\lib\site-packages\pandas\core\indexing.py:1843: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value


,cell_type,source,id
cell_id,,,
1862f0a6,code,# This Python 3 environment comes with many helpful analytics libraries installed # It is defined by the kaggle/pyth...,00001756c60be8
2a9e43d6,code,"import numpy as np import pandas as pd import random from sklearn.model_selection import train_test_split, cross_va...",00001756c60be8
038b763d,code,import warnings warnings.filterwarnings('ignore'),00001756c60be8
2eefe0ef,code,matplotlib.rcParams.update({'font.size': 14}),00001756c60be8
0beab1cd,code,"def evaluate_preds(train_true_values, train_pred_values, test_true_values, test_pred_values): print(""Train R2:\t...",00001756c60be8
9a78ab76,code,TRAIN_DATASET_PATH = '/kaggle/input/real-estate-price-prediction-moscow/train.csv' TEST_DATASET_PATH = '/kaggle/inpu...,00001756c60be8
ebe125d5,code,train_df = pd.read_csv(TRAIN_DATASET_PATH) train_df.tail(),00001756c60be8
d9dced8b,code,train_df.dtypes,00001756c60be8
86497fe1,code,num_feat = list(train_df.select_dtypes(exclude='object').columns) obj_feat = list(train_df.select_dtypes(include='ob...,00001756c60be8


In [11]:
def summary(code_cell):
    code_cell = code_cell.replace("\n" , " ")
    inputs_ids = tokenizer(code_cell , return_tensors="pt")["input_ids"]
    generated_ids = model.generate(inputs_ids , max_length=50)

    return tokenizer.decode(generated_ids[0], skip_special_tokens=True)

from transformers import RobertaTokenizer , T5ForConditionalGeneration
tokenizer = RobertaTokenizer.from_pretrained('Salesforce/codet5-base-multi-sum')
model = T5ForConditionalGeneration.from_pretrained('Salesforce/codet5-base-multi-sum')

# allCode["summ_code"] = allCode["source"].apply(summary)
allCode = allCode.assign(summary=allCode["source"].apply(summary))
allCode

Token indices sequence length is longer than the specified maximum sequence length for this model (2466 > 512). Running this sequence through the model will result in indexing errors


,cell_type,source,id,summary
cell_id,,,,
1862f0a6,code,# This Python 3 environment comes with many helpful analytics libraries installed # It is defined by the kaggle/pyth...,00001756c60be8,This environment is used to load the analytics library
2a9e43d6,code,"import numpy as np import pandas as pd import random from sklearn.model_selection import train_test_split, cross_va...",00001756c60be8,Imports the sklearn library and creates a figure with the model selection and cross - validation.
038b763d,code,import warnings warnings.filterwarnings('ignore'),00001756c60be8,Ignore warnings.
2eefe0ef,code,matplotlib.rcParams.update({'font.size': 14}),00001756c60be8,Update the font size of the rcParams
0beab1cd,code,"def evaluate_preds(train_true_values, train_pred_values, test_true_values, test_pred_values): print(""Train R2:\t...",00001756c60be8,Evaluate prediction on the training and test data.
9a78ab76,code,TRAIN_DATASET_PATH = '/kaggle/input/real-estate-price-prediction-moscow/train.csv' TEST_DATASET_PATH = '/kaggle/inpu...,00001756c60be8,This function sets the path to the train and test data sets.
ebe125d5,code,train_df = pd.read_csv(TRAIN_DATASET_PATH) train_df.tail(),00001756c60be8,Read the train data set from the train_dataset_path and return the last row in the train data set.
d9dced8b,code,train_df.dtypes,00001756c60be8,train_df. dtypes
86497fe1,code,num_feat = list(train_df.select_dtypes(exclude='object').columns) obj_feat = list(train_df.select_dtypes(include='ob...,00001756c60be8,Number of features in training data frame


因为查阅发现codeT5模型本身就在英文自然语言上训练过，因此直接使用该模型对所有代码内容进行总结压缩

In [12]:
code_nb_summ = allCode.groupby("id")["summary"].apply(lambda x: x.str.cat(sep=". ")).values #拼接每一个代码快的summary
code_nb_summ[0]

'This environment is used to load the analytics library. Imports the sklearn library and creates a figure with the model selection and cross - validation.. Ignore warnings.. Update the font size of the rcParams. Evaluate prediction on the training and test data.. This function sets the path to the train and test data sets.. Read the train data set from the train_dataset_path and return the last row in the train data set.. train_df. dtypes. Number of features in training data frame. Read the test data set and return the last row.. Print the training and test data.. Check if the training and test dataframes have the same shape.. Sample submission from Moscow. Convert the id to str. Plot a histogram of the training data.. Describe the training data.. Create a joint plot of the rooms and price.. Create a joint plot of the KitchenSquare and Price.. Create a joint plot of the KitchenSquare and the price.. Plots a histogram of the target distribution count and price.. Plots correlation betwee

In [13]:
code_nb_summ = summary(code_nb_summ[0])
code_nb_summ

'This environment is used to load the analytics library and create a figure with the model selection and cross - validation'

最后发现效果不错

### 正式开始，最终决定使用第二种方案

In [5]:
import json
from pathlib import Path
import numpy as np
import pandas as pd
import torch
# from scipy import sparse
from tqdm import tqdm

pd.options.display.width = 180
pd.options.display.max_colwidth = 120
MAX_FILE_NUM = 1000

读取json文件的函数 read_notebook

In [6]:
def read_notebook(path):
    df = pd.read_json(
            path,
            dtype={'cell_type':'category' , 'source':'str'}
    ).assign(id=path.stem).rename_axis('cell_id')
    return df

In [7]:
paths_train = list(Path('train').glob('*json'))[: MAX_FILE_NUM]
paths_train

[WindowsPath('train/00001756c60be8.json'),
 WindowsPath('train/00015c83e2717b.json'),
 WindowsPath('train/0001bdd4021779.json'),
 WindowsPath('train/0001daf4c2c76d.json'),
 WindowsPath('train/0002115f48f982.json'),
 WindowsPath('train/00035108e64677.json'),
 WindowsPath('train/00038c2941faa0.json'),
 WindowsPath('train/00039fc77a9dd1.json'),
 WindowsPath('train/00058dc97eae4b.json'),
 WindowsPath('train/000597ac4c6700.json'),
 WindowsPath('train/000624d747afd3.json'),
 WindowsPath('train/00062ab8487156.json'),
 WindowsPath('train/000757b90aaca0.json'),
 WindowsPath('train/0007f21ee357b5.json'),
 WindowsPath('train/000890decea38e.json'),
 WindowsPath('train/0008ba887b3817.json'),
 WindowsPath('train/0009acaa9aa47e.json'),
 WindowsPath('train/000a2f5243e1ca.json'),
 WindowsPath('train/000a4651cce8f4.json'),
 WindowsPath('train/000af347c31e72.json'),
 WindowsPath('train/000b8e6d58544b.json'),
 WindowsPath('train/000bbb79a2fe3c.json'),
 WindowsPath('train/000c0a9b2fef4d.json'),
 WindowsPat

## 读取数据进入dataFrame

In [8]:
notebook_train = [
    read_notebook(path) for path in tqdm(paths_train , desc = 'Train NBs')
]
df = (
    pd.concat(notebook_train)
    .set_index('id' , append=True)
    .swaplevel()
    .sort_index(level='id' , sort_remaining=False)
)

df

Train NBs: 100%|██████████| 1000/1000 [00:03<00:00, 261.92it/s]


cell_type                                                                                                                   source
id             cell_id                                                                                                                                    
00001756c60be8 1862f0a6      code  # This Python 3 environment comes with many helpful analytics libraries installed\n# It is defined by the kaggle/pyt...
               2a9e43d6      code  import numpy as np\nimport pandas as pd\nimport random\n\nfrom sklearn.model_selection import train_test_split, cros...
               038b763d      code                                                                       import warnings\nwarnings.filterwarnings('ignore')
               2eefe0ef      code                                                                            matplotlib.rcParams.update({'font.size': 14})
               0beab1cd      code  def evaluate_preds(train_true_values, train_pred_values, test_true_values, test_pred_values):\n    print("Train R2:\...
...                           ...                                                                                                                      ...
01e495df921c7e e741ca70  markdown                                                                                                  # Player influence Area
               365a2ea3  markdown  In this notebook, I code the **VIP hint** given by @MichaelLopez. You can find the paper at : http://www.lukebornn.c...
               32ced18c  markdown                                                                                                                # Imports
               36c8eb2c  markdown                                                                                                        ### Class Wrapper
               ee9c3788  markdown  Quoting from the paper : based on this, we present a pitch control model that summarizes the level of influence of e...

[46514 rows x 2 columns]

## 对代码片段部分生成summary

In [9]:
df_code = df[df["cell_type"] == "code"]
df_code

cell_type                                                                                                                   source
id             cell_id                                                                                                                                    
00001756c60be8 1862f0a6      code  # This Python 3 environment comes with many helpful analytics libraries installed\n# It is defined by the kaggle/pyt...
               2a9e43d6      code  import numpy as np\nimport pandas as pd\nimport random\n\nfrom sklearn.model_selection import train_test_split, cros...
               038b763d      code                                                                       import warnings\nwarnings.filterwarnings('ignore')
               2eefe0ef      code                                                                            matplotlib.rcParams.update({'font.size': 14})
               0beab1cd      code  def evaluate_preds(train_true_values, train_pred_values, test_true_values, test_pred_values):\n    print("Train R2:\...
...                           ...                                                                                                                      ...
01e495df921c7e ed6cb193      code  # select a random play to be plotted\nmy_play_id = choice(dominance_df['PlayId'].unique())           # if you want a...
               03a4b95a      code  front = 15\nbehind = 5\nleft = right = 20\nnum_points_meshgr = (30, 15)   # don't make it too large otherwise it'll ...
               7416a784      code  class Controller:\n    '''This class is a wrapper for the two functions written above'''\n    def __init__(self, pla...
               cbfd2f33      code  # Example: how to use the class?\ncontrol = Controller(my_play)\ncoords = my_play.iloc[1, 54:56].values         # le...
               c858ecab      code                                                                                                control.display_control()

[30613 rows x 2 columns]

准备清洗数据的函数以及用到的模型

In [19]:
from transformers import RobertaTokenizer , T5ForConditionalGeneration
tokenizer = RobertaTokenizer.from_pretrained('Salesforce/codet5-base-multi-sum')
model = T5ForConditionalGeneration.from_pretrained('Salesforce/codet5-base-multi-sum' , max_length=512)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

def summary(code_cell):
    code_cell = code_cell.replace("\n" , " ") #去除 \n 符号
    inputs_ids = tokenizer(code_cell , return_tensors="pt")["input_ids"] # 转码
    inputs_ids = inputs_ids.to(device)
    generated_ids = model.generate(inputs_ids , max_length=50) # 生成summary

    return tokenizer.decode(generated_ids[0], skip_special_tokens=True) # 解码

RuntimeError: CUDA out of memory. Tried to allocate 96.00 MiB (GPU 0; 6.00 GiB total capacity; 4.62 GiB already allocated; 0 bytes free; 4.72 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

试一下数据处理的速度

In [15]:
import datetime
df_demo = read_notebook(paths_train[1])
allCode = df_demo[(df_demo["cell_type"] == "code").tolist()]
allCode

,cell_type,source,id
cell_id,,,
c417225b,code,"import numpy as np # linear algebra\nimport pandas as pd\npd.set_option(""display.max_rows"", 101)\nimport os\nprint(o...",00015c83e2717b
51e3cd89,code,"input_dir = ""../input/""",00015c83e2717b
2600b4eb,code,"train_df = pd.read_csv(""../input/train.csv"")\nsample_df = pd.read_csv(""../input/sample_submission.csv"")",00015c83e2717b
75b65993,code,class_dict = defaultdict(int)\n\nkind_class_dict = defaultdict(int)\n\nno_defects_num = 0\ndefects_num = 0\n\nfor co...,00015c83e2717b
cf195f8b,code,"print(""the number of images with no defects: {}"".format(no_defects_num))\nprint(""the number of images with defects: ...",00015c83e2717b
...,...,...,...
b8f3850a,code,test_final = test_df.loc[test_df['prediction'] <0.50],00015c83e2717b
ccbe6713,code,test_final.describe(),00015c83e2717b
1ecd4e35,code,test_df,00015c83e2717b


## 这个方法不现实，cuda内存不支持这么搞

In [16]:
import datetime
start = datetime.datetime.now()
df_code = df_code.assign(summary=df_code["source"].apply(summary))
end = datetime.datetime.now()
print('平均每行需要时间：' , (end-start)/df_code.__len__())
df_code


Token indices sequence length is longer than the specified maximum sequence length for this model (2466 > 512). Running this sequence through the model will result in indexing errors


RuntimeError: CUDA out of memory. Tried to allocate 280.00 MiB (GPU 0; 6.00 GiB total capacity; 4.40 GiB already allocated; 0 bytes free; 4.67 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF